
# System Q&A z Fine-Tuningiem T5 i Retrieverem (FAISS)

**Cel zadania:**  
Stwórz system pytanie-odpowiedź (Q&A), który:
- Wykorzystuje **Twój własny zbiór danych** (CSV: pytania, konteksty, odpowiedzi),
- Trenuje model **T5-small** na Twoim zbiorze (fine-tuning),
- Wyszukuje kontekst za pomocą **retrievera FAISS**,
- Generuje odpowiedzi z pomocą fine-tunowanego modelu.

---

## Zadania (3 punkty):
1) Przygotowanie własnych danych (CSV)  
2) Fine-tuning modelu T5-small  
3) Stworzenie retrievera FAISS + generowanie odpowiedzi  
4) Testy i analiza wyników  
5) Raport PDF z wnioskami

Fine tuning = 1 punkt   
RAG = 1 punkt   
PDF = 1 punkt   

**Uwaga:** Fine-tuning powinien być wykonany na **innym temacie lub zakresie pytań** niż dokumenty w retrieverze FAISS.
Nie używaj tych samych danych w obu miejscach!


##  Przygotowanie danych

Przygotuj własny plik CSV z danymi: `question`, `context`, `answer`.

Przykładowy format (przygotuj samodzielnie!):

| question                      | context                                    | answer       |
|------------------------------|--------------------------------------------|--------------|
| What is the capital of France?| The capital of France is Paris.             | Paris        |
| What is the currency of USA? | The currency of the USA is the dollar.      | The dollar   |

In [ ]:
!pip install langchain-community faiss-cpu
#!pip install transformers datasets sentence-transformers

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
import faiss
import numpy as np

In [ ]:
# Wczytaj swój plik CSV
df = pd.read_csv("your_data.csv")
df.head()


## Fine-tuning modelu T5-small
**Uwaga:** Fine-tuning powinien być wykonany na **innym temacie lub zakresie pytań** niż dokumenty w retrieverze FAISS.
Nie używaj tych samych danych w obu miejscach!

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [ ]:
dataset = Dataset.from_pandas(df)

def preprocess_function(examples):
    inputs = [f"question: {q} context: {c}" for q, c in zip(examples['question'], examples['context'])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["answer"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="./t5-finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=100,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

In [ ]:
# Sprawdź czy model jest w stanie odpowiedzieć na pytania z Twojego zbioru danych!
# Możesz zmodyfikować parametry treningu


## Retriever (FAISS) + Generowanie odpowiedzi

Zbuduj system, który wyszukuje kontekst w dokumentach i generuje odpowiedzi.

Przygotuj własną listę dokumentów do wyszukiwania (minimum 20 zdań).

**Uwaga:** Twoja baza wiedzy (lista dokumentów) w retrieverze **nie powinna być taka sama jak dane do fine-tuningu**.
Celem jest sprawdzenie, jak system radzi sobie z nowymi pytaniami i nową bazą wiedzy.

In [ ]:
# Przykład, stwórz własny zbiór!
documents = [
    "Paris is the capital of France.",
    "The dollar is the currency of the USA.",
    "Python is a programming language that lets you work quickly."
]

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
document_embeddings = [embedding_model.embed_query(doc) for doc in documents]
document_embeddings_np = np.array(document_embeddings).astype('float32')

index = faiss.IndexFlatL2(document_embeddings_np.shape[1])
index.add(document_embeddings_np)

In [ ]:
def generate_answer(query, top_k=2):
    query_embedding = embedding_model.embed_query(query)
    query_embedding_np = np.array(query_embedding).reshape(1, -1).astype('float32')

    distances, indices = index.search(query_embedding_np, top_k)
    retrieved_docs = " ".join([documents[i] for i in indices[0]])

    input_text = f"question: {query} context: {retrieved_docs}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    output = model.generate(input_ids)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer

In [ ]:
# Przykładowe testy (zmień pytania na swoje)
questions = ["What is the capital of France?", "What is the currency of USA?"]

for question in questions:
    answer = generate_answer(question)
    print(f"Q: {question}\nA: {answer}\n")

## Raport PDF
**Twoje zadanie:** Przygotuj krótki raport PDF (1-2 strony) z opisem:
- Jak przygotowałeś dane?
- Jak wyglądały wyniki?
- Jak działa RAG?  
- Jakie widzisz ograniczenia RAG?  
- Jakie są Twoje refleksje po eksperymencie z fine-tuningiem?   
- Jak zmieniły się odpowiedzi po fine-tuningu?
- Jak zmiana parametru k może wpłynąć na wyniki?